In [14]:
import pandas as pd
import json
import csv
import requests
from collections import Counter


class Assesment:
    def __init__(self, store_path):
        self.store_path=store_path
        self.base_df = pd.DataFrame()
        self.first_df = None
        self.second_df= None
        
    def read_file(self):
        """read the file"""
        self.base_df=pd.read_json(self.store_path)
        return self.base_df
    def test_1(self):
        """ sort the label and count the occurences of labels"""
        df_sorted=self.base_df['maker_response']['sensor_fusion_v2']['data']['tracks']
        li=[]
        for i in df_sorted:
            li.append(i['label'])
        d = {i:li.count(i) for i in li}
        self.first_df = pd.DataFrame(list(d.items()), columns = ['label', 'count'])
        
    def excel_file(self):
        """ save the mutliple dataframes into a single excel sheet consiting of multiple sheets in excel"""
        if not self.first_df.empty and not self.second_df.empty:
            with pd.ExcelWriter('final_output.xlsx') as writer:
                self.first_df.to_excel(writer, sheet_name='Output_1', index=False)
                self.second_df.to_excel(writer, sheet_name='Output_2', index=False)
        print("Successfully created the file")
        
    def test_2(self):
        """ Create a dataframe consists of  ['label', 'attribute_name', 'attribute_value', 'attribute_count']"""
        data=[]
        for building in self.base_df['maker_response']['sensor_fusion_v2']['data']['annotations']:
            field_name=building["label"]
            for key,val in building['attributes'].items():
                attribute_name=key
                attribute_value=val['value']
                data.append({'label': field_name, 'attribute_name': attribute_name, 'attribute_value': attribute_value})
        data_df = pd.DataFrame(data)
        data_df['label'] = data_df['label'].apply(str)
        data_df['attribute_name'] = data_df['attribute_name'].apply(str)
        data_df['attribute_value'] = data_df['attribute_value'].apply(str)
        self.second_df=data_df.groupby(['label', 'attribute_name', 'attribute_value']).value_counts().reset_index(name='attribute_count')
        self.second_df
        
    def save(self):
        self.read_file()
        self.test_1()
        self.test_2()
        self.excel_file()
        
if __name__ == '__main__':
    '''Provide the path or url '''
    store_path="C:\\Users\\admin\\Downloads\\build.json"
    faObj = Assesment(store_path)
    faObj.save()
        

Successfully created the file
